## Sparse Grid Regression with the Combination Technique: 
This notebook provides an overview on how to use the Regression with the (spatially adaptve) Combination Technique. The complexity of the computation is lowered while still providing accurate results compared to full grids. The class $\texttt{Regression}$ implements those functionalities. Its methods $\texttt{train}$, $\texttt{optimize\_coefficients}$ and $\texttt{test}$ encapsulate the most important steps.

First, an object of class $\texttt{Regression}$ has to be initialized. The data set and its target values have to be given in the constructor. Additionally, a regularization parameter $\lambda$ and the choice for the regularization matrix (C or I) have to be determined. Additionally, the range to which the data is scaled to can be changed. The standard value is $[0.05,0.95]$.

In the next step, the regression object can be trained. The method $\texttt{train}$ takes 4 parameters. The percentage of testdata decides, how the data set given in the constructor is split (training, validation and test). Minimum and maximum level have to be given for the Combination Technique and with the last boolean parameter, white noise can be added to the target values of the training and validation data. The $\texttt{train}$ method returns a combiObject. With this, predictions of new data points can be made.

As a next, optional step, the combination technique can be optimized (Opticom). Therefore, the method $\texttt{optimize\_coefficients}$ of Regression has to be called. It needs the combiObject from the previous method and the choice of option: 1 -> Garcke approach, 2 -> least squares based, 3 -> error based approach.

The last step is to test the computed function with the help of $\texttt{test}$. It again needs the combiObject and it computes the mean squared error of the test data. 

### Regression with the normal combination technique:

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
import math

def construct_dataset(dim, function, num_points):
    grid_points = np.random.rand(num_points,dim)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()

dim = 2                              
number_samples = 300                 
regularization_matrix = 'C'          # other possibility: 'I'
regularization_parameter = 10.**-6   # parameter lambda -> prevent overfitting

# set up function and draw samples
function = GenzGaussian((0.5,0.5), (15,15))
data, target = construct_dataset(dim, function, number_samples)

# initialize regression object
operation = Regression(data, target, regularization_parameter, regularization_matrix)


print("Plot of the data samples: ")
operation.plot_dataset()
print("Plot of the function: ")
function.plot((0,0),(1,1))

# train the regression object
combiObject = operation.train(percentage_of_testdata=0.2, minimum_level=1, maximum_level=4, noisy_data=True)
# print combination scheme with partial results
combiObject.print_resulting_combi_scheme(operation=operation)
# print resulting sparse grid
print("Sparse Grid:")
combiObject.print_resulting_sparsegrid(markersize=20)
# print combined result
print("Plot of Regression:")
combiObject.plot(contour=True)
# calculate current error (without Opticom)
print("Testing error without Opticom: ")
print(operation.test(combiObject))
print("Time used: ")
print(combiObject.get_time_used())
# perform Opticom (option can be changed)
operation.optimize_coefficients(combiObject, option=2)
# calculate error after Opticom
print("Testing error with Opticom: ")
print(operation.test(combiObject))

### Regression with the spatially adaptive combination technique:

The same steps as above can be performed. For the training methods, the following parameters can be specified: 
- percentage of test data: how many data points are used for testing
- margin: at what points will be refined (at those with error >= margin*max_error 
- tolerance: stopping criterion depending on error
- max_evaluations: refinement stops when current evaluations >= max_evalauations
- do_plot: specifiy whether each refinement step is plotted
- noisy_data: decide whether white noise is added to the training target values

In [ ]:
# import sparseSpACE
%matplotlib inline
import numpy as np
from sparseSpACE.ErrorCalculator import *
from sparseSpACE.GridOperation import *
from sparseSpACE.StandardCombi import *
from sparseSpACE.DEMachineLearning import *
import math

def construct_dataset(dim, function, num_points):
    grid_points = np.random.rand(num_points,dim)
    y_vals = np.array([function(x) for x in grid_points])
    return grid_points, y_vals.flatten()


dim = 2                              
number_samples = 300                 
regularization_matrix = 'C'          # other possibility: 'I'
regularization_parameter = 10.**-6   # parameter lambda -> prevent overfitting

# set up function and draw samples
function = GenzGaussian((0.5,0.5), (20,20))
data, target = construct_dataset(dim, function, number_samples)

# initialize regression object
operation = Regression(data, target, regularization_parameter, regularization_matrix)


print("Plot of the data samples: ")
operation.plot_dataset()
print("Plot of the function: ")
function.plot((0,0),(1,1))
# train the regression object
adaptiveCombiInstanceSingleDim = operation.train_spatially_adaptive(percentage_of_testdata=0.2, margin=0.7, tolerance=10.**-5, max_evaluations=18, do_plot=False, noisy_data=False)
# print combination scheme 
adaptiveCombiInstanceSingleDim.print_resulting_combi_scheme(operation=operation)
# print resulting sparse grid
print("Sparse Grid:")
adaptiveCombiInstanceSingleDim.print_resulting_sparsegrid(markersize=20)
# print combined result
print("Plot of Regression:")
adaptiveCombiInstanceSingleDim.plot(contour=True)
# calculate error without Opticom
print("Testing error without Opticom: ")
print(operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim))
# perform Opticom (option can be changed)
operation.optimize_coefficients_spatially_adaptive(adaptiveCombiInstanceSingleDim, option=2)
# calculate error without Opticom
print("Testing error with Opticom: ")
print(operation.test_spatially_adaptive(adaptiveCombiInstanceSingleDim))